# SEIR Bimodal
This Jupyter notebook presents the SEIR Bimodal model EDOs implementation in python.
The equations are the following: 

\begin{align}
    \dot{S_s} &=-\frac{\beta\alpha_s S_s}{N}\left(I_s + I_r\right) \\
    \dot{S_r} &=-\frac{\beta\alpha_s S_r}{N}\left(I_s + I_r\right)-\frac{\beta\alpha_r S_r}{p_r N} I_r \\ 
    \dot{E_s} &=\frac{\beta\alpha_s S_s}{N}\left(I_s + I_r\right) - \sigma E_s \\
    \dot{E_r} &=\frac{\beta\alpha_s S_r}{N}\left(I_s + I_r\right) + \frac{\beta\alpha_r S_r}{p_r N} I_r -  \sigma E_{s}\\     
    \dot{I_s} &=\sigma E_s - \gamma I_s \\
    \dot{I_r} &=\sigma E_r - \gamma I_r \\
    \dot{R}   &=\gamma (I_s + I_r) \\
\end{align}



In [1]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR_bimodal/')
sys.path.insert(1, 'src/SEIR_bimodal/')
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, 'src/SEIR/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/utils/')
sys.path.insert(1, '../')

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    %matplotlib tk
    print('Mac')

from datetime import datetime
from datetime import timedelta
from joblib import Parallel, delayed
import multiprocessing

import numpy as np
import pandas as pd
import requests

from class_SEIR import SEIR
from class_SEIR_bimodal import SEIRbimodal
from Quarantine import Quarantine
from SEIR_parallel import seirMetaAnalysis

Linux


## Define Simulation Parameters

In [31]:
beta = 0.4
mu = 1 
# Incubation rate
gamma = 0.2
# Recovery rate
sigma = 0.1

# Risk population proportion
p_r = 0.05

# Total population
population = 1000000

# Saturation Dynamics:
k = 0

# simulation time
tsim = 1000

# Do exposed infect?
expinfection = 0 # NO =)

### Mobility functions

In [32]:
# Mobility
mob_s = 0.55
mob_r = 0

alpha_s = Quarantine(mob_s).alpha
alpha_r = Quarantine(mob_r).alpha


### Initial conditions

In [33]:
I0 = 200
I_ac0 = 0
I_d0 = 0

### Create model and integrate

In [34]:
seirbimodal = SEIRbimodal(tsim,alpha_s,alpha_r,beta,mu,sigma = sigma,gamma = gamma,p_r=p_r,k=k,I0=I0,I_ac0=I_ac0,I_d0=I_d0,population=population,expinfection = expinfection )
seirbimodal.integr_sci(0,tsim,0.1)


  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 181
     njev: 6
      nlu: 6
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 7.90569415e-07, 1.58113883e-06, 6.44039819e-03,
       1.28792152e-02, 1.93180323e-02, 8.37062029e-02, 1.48094373e-01,
       2.12482544e-01, 2.76870715e-01, 5.59370515e-01, 8.41870316e-01,
       1.12437012e+00, 1.40686992e+00, 1.68936972e+00, 2.14021594e+00,
       2.59106217e+00, 3.04190839e+00, 3.49275461e+00, 3.94360084e+00,
       4.49506348e+00, 5.04652612e+00, 5.59798877e+00, 6.14945141e+00,
       6.71780273e+00, 7.28615405e+00, 7.85450536e+00, 8.79109672e+00,
       9.72768808e+00, 1.06642794e+01, 1.19381903e+01, 1.32121011e+01,
       1.44860119e+01, 1.63607963e+01, 1.82355806e+01, 2.01103650e+01,
       2.10045776e+01, 2.18987902e+01, 2.27930029e+01, 2.57671734e+01,
       2.87413439e+01, 3.17155145e+01, 3.86467793e+01, 4.55780440e+01,
       5.25093088e+01, 6.52841425e+01, 7

# Analysis and QA

## People conservation

In [28]:
plt.plot(seirbimodal.t,seirbimodal.S+seirbimodal.E+seirbimodal.I+seirbimodal.R,label='Simulation total people')
plt.hlines(population,0,tsim,label='Total People') 
plt.title('Population QA')
plt.legend(loc=0)
plt.show()

## SEIR Dynamics

In [35]:
# Plot SEIR
plt.plot(seirbimodal.t,seirbimodal.S,label='S')
plt.plot(seirbimodal.t,seirbimodal.E,label='E')
plt.plot(seirbimodal.t,seirbimodal.I,label='I')
plt.plot(seirbimodal.t,seirbimodal.R,label='R')
plt.title('SEIR')
plt.legend(loc=0)
plt.show()

## Bimodal Dynamics

In [23]:
plt.plot(seirbimodal.t,seirbimodal.Is,label='Is')
plt.plot(seirbimodal.t,seirbimodal.Ir,label='Ir')
plt.title('Active Infected')
plt.legend(loc=0)
plt.show()

# SEIRbimodal vs SEIR

In [48]:
p_r = 0.05
# Mobility
mob_s = 0.8
mob_r = 1-mob_s 0.5

alpha = Quarantine(mob_s).alpha
alpha_s = Quarantine(mob_s).alpha
alpha_r = Quarantine(mob_r).alpha

## Simulate SEIRbimodal

In [49]:
seirbimodal = SEIRbimodal(tsim,alpha_s,alpha_r,beta,mu,sigma = sigma,gamma = gamma,p_r=p_r,k=k,I0=I0,I_ac0=I_ac0,I_d0=I_d0,population=population,expinfection = expinfection )
seirbimodal.integr_sci(0,tsim,0.1)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 402
     njev: 13
      nlu: 13
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 7.90569415e-07, 1.58113883e-06, 6.77114728e-03,
       1.35407134e-02, 2.03102796e-02, 8.80059410e-02, 1.55701602e-01,
       2.23397264e-01, 2.91092925e-01, 5.86965744e-01, 8.82838563e-01,
       1.17871138e+00, 1.47458420e+00, 1.77045702e+00, 2.22256073e+00,
       2.67466443e+00, 3.12676814e+00, 3.57887185e+00, 4.03097555e+00,
       4.58471624e+00, 5.13845693e+00, 5.69219762e+00, 6.24593832e+00,
       6.80820537e+00, 7.37047241e+00, 7.93273946e+00, 9.04545560e+00,
       1.01581717e+01, 1.12708879e+01, 1.26861984e+01, 1.41015089e+01,
       1.55168194e+01, 1.71904428e+01, 1.88640662e+01, 2.05376896e+01,
       2.18468774e+01, 2.31560652e+01, 2.44652529e+01, 2.53193122e+01,
       2.61733715e+01, 2.70274308e+01, 2.79900436e+01, 2.89526565e+01,
       2.99152693e+01, 3.20499746e+01,

## Simulate SEIR

In [50]:
seir = SEIR(tsim,alpha,beta,mu,sigma = sigma, gamma = gamma, k=k,I=I0,I_ac=I_ac0,I_d=I_d0,population=population,expinfection = expinfection)
seir.integr_sci(0,tsim,0.1)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 380
     njev: 18
      nlu: 18
      sol: None
   status: 0
  success: True
        t: array([0.00000000e+00, 7.90569415e-07, 1.58113883e-06, 5.68159864e-03,
       1.13616161e-02, 1.70416336e-02, 7.38418086e-02, 1.30641984e-01,
       1.87442159e-01, 2.44242334e-01, 5.20145925e-01, 7.96049517e-01,
       1.07195311e+00, 1.34785670e+00, 1.62376029e+00, 2.07460500e+00,
       2.52544971e+00, 2.97629442e+00, 3.42713912e+00, 3.87798383e+00,
       4.42947496e+00, 4.98096609e+00, 5.53245722e+00, 6.08394834e+00,
       6.65430313e+00, 7.22465792e+00, 7.79501271e+00, 8.66120159e+00,
       9.52739048e+00, 1.03935794e+01, 1.15867122e+01, 1.27798451e+01,
       1.39729779e+01, 1.57359543e+01, 1.74989307e+01, 1.92619071e+01,
       2.15377041e+01, 2.38135012e+01, 2.60892983e+01, 2.92476697e+01,
       3.00372626e+01, 3.08268554e+01, 3.16164483e+01, 3.21338001e+01,
       3.26511519e+01, 3.31685037e+01,

## SEIR Bimodal Dynamics

In [51]:
plt.plot(seirbimodal.t,seirbimodal.Is,label='Is')
plt.plot(seirbimodal.t,seirbimodal.Ir,label='Ir')
plt.title('Bimodal Dynamics Active Infected')
plt.legend(loc=0)
plt.show()

## Global Dynamics comparison

In [52]:
plt.plot(seirbimodal.t,seirbimodal.S,label='S_bi',color='blue')
plt.plot(seirbimodal.t,seirbimodal.E,label='E_bi',color='cyan')
plt.plot(seirbimodal.t,seirbimodal.I,label='I_bi',color='red')
plt.plot(seirbimodal.t,seirbimodal.R,label='R_bi',color='green')
plt.plot(seir.t,seir.S,label='S',color='blue',linestyle = 'dashed')
plt.plot(seir.t,seir.E,label='E',color='cyan',linestyle = 'dashed')
plt.plot(seir.t,seir.I,label='I',color='red',linestyle = 'dashed')
plt.plot(seir.t,seir.R,label='R',color='green',linestyle = 'dashed')
plt.title('SEIR')
plt.legend(loc=0)
plt.show()

In [16]:
plt.plot(seirbimodal.t,seirbimodal.Is_d,label='Is')
plt.plot(seirbimodal.t,seirbimodal.Ir_d,label='Ir')
plt.plot(seirbimodal.t,seirbimodal.Ir_d+seirbimodal.Is_d,label='Itot')
plt.plot(seir.t,seir.I_d,label='I',color='red',linestyle = 'dashed')
plt.title('Bimodal Dynamics Active Infected')
plt.legend(loc=0)
plt.show()